In [1]:
from langchain.document_loaders import PDFPlumberLoader, DirectoryLoader
from langchain.document_loaders.word_document import Docx2txtLoader
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import os

cached_db = False
loader_type = "docx"
loader = None

print(Chroma)
if not cached_db:
    if loader_type == "pdf":
        loader_cls, glob = PDFPlumberLoader, "*.pdf"
    elif loader_type == "docx":
        loader_cls, glob = Docx2txtLoader, "*.docx"
    elif loader_type == "txt":
        loader_cls, glob = UnstructuredFileLoader, "*.txt"
    else:
        loader_cls, glob = None, None
    loader = DirectoryLoader(
        "data/MGRP", glob=glob, loader_cls=loader_cls, show_progress=True
    )
    pages = loader.load_and_split()

<class 'langchain.vectorstores.chroma.Chroma'>


100%|██████████| 10/10 [00:00<00:00, 79.79it/s]


In [2]:
%set_env PYTORCH_CUDA_ALLOC_CONF=backend:cudaMallocAsync

import importer
from langchain.vectorstores import Chroma
from langchain.embeddings.spacy_embeddings import SpacyEmbeddings
import tiktoken
from langchain_extras.llms.exllama import ExLlama, BasicStreamingHandler
from langchain.chains import loading, RetrievalQA

handler = BasicStreamingHandler()

encoding = tiktoken.get_encoding("cl100k_base")
total_token_count = 0
model_directory = "models/TheBloke_GPT4All-13B-Snoozy-SuperHOT-8K-GPTQ"

print(f"Number of pages: {len(pages)}")

llm = ExLlama(
    streaming=True,
    model_path=model_directory,
    lora_path=None,
    temperature=0.7,
    beams=1,
    beam_length=40,
    stop_sequences=["Human:", "User:", "AI:"],
    callbacks=[handler],
    verbose=False,
    max_seq_len = 4096,
    alpha_value=4.0,  # For use with any models
    compress_pos_emb=4.0,  # For use with superhot
    # set_auto_map = "3, 2" #Gpu split, this will split 3gigs/2gigs
)


def tiktoken_length(text):
    global total_token_count
    token_count = encoding.encode(text)
    if len(token_count) == 0:
        return 0
    else:
        total_token_count += token_count[0]
        return token_count[0]

texts = [p.page_content for p in pages]

wizard_templte = """You are a helpful AI Assistant. 
{history}
### HUMAN: {input}
### ASSISTANT: """

embeddings = SpacyEmbeddings()

if cached_db:
    db = Chroma(persist_directory="data", embedding_function=embeddings)
else:
    db = Chroma.from_documents(pages, embeddings, persist_directory="data")
    db.persist()

qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=db.as_retriever(), verbose=True
)
    
qa.save("data/chains/mgrp_chain.yaml")
handler.set_chain(qa.combine_documents_chain)
print("\n")


env: PYTORCH_CUDA_ALLOC_CONF=backend:cudaMallocAsync
/home/jovyan/work/importer.py
Number of pages: 87


2023-07-10 18:29:49.274490: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 18:29:49.405150: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-10 18:29:49.921291: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-10 18:29:49.921378: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [3]:
qa.run("Can you summarize the story?")



> Entering new  chain...
 Yes, here's a summary of the story so far: After Tama dies from being killed by Swim, who becomes the new Master, Ripple tries to kill her former comrades one by one using their weapons. However, they are able to survive due to having been given special powers by Ruler before she died. They eventually find a way to defeat Ripple with the help of a magical phone and return to normal life. Meanwhile, Swim continues to be the Master while also trying to keep her identity hidden from others.
> Finished chain.


" Yes, here's a summary of the story so far: After Tama dies from being killed by Swim, who becomes the new Master, Ripple tries to kill her former comrades one by one using their weapons. However, they are able to survive due to having been given special powers by Ruler before she died. They eventually find a way to defeat Ripple with the help of a magical phone and return to normal life. Meanwhile, Swim continues to be the Master while also trying to keep her identity hidden from others."